# Финальный проект. Анализ оттока клиентов банка

# Материалы

**Презентация** - https://disk.yandex.ru/i/f6QX_aULG2yF4w

**Дашборд** - https://public.tableau.com/app/profile/grigoriy3340/viz/___16618826404430/Dashboard1?publish=yes

**Описание проекта:**

Необходимо проанализировать клиентов регионального банка и выделить портрет клиентов, которые склонны уходить из банка.

**Описание данных:**

- `userid` — идентификатор пользователя,
- `score` — баллы кредитного скоринга,
- `City` — город,
- `Gender` — пол,
- `Age` — возраст,
- `Objects` — количество объектов в собственности,
- `Balance` — баланс на счёте,
- `Products` — количество продуктов, которыми пользуется клиент,
- `CreditCard` — есть ли кредитная карта,
- `Loyalty` — активный клиент,
- `estimated_salary` — заработная плата клиента,
- `Churn` — ушёл или нет.

**Декомпозация (задачи проекта):**

1. Загрузка данных и первый взгляд.
2. Предобработка данных:
 - приведение названий строк/столбцов к единому регистру;
 - проверка на пропуски, дубликаты.
3. Исследовательский анализ данных EDA.
4. Портрет клиентов, склонных уходить из банка.
5. Формулировка и проверка гиппотез:
 - гипотеза различия дохода между теми клиентами, которые ушли и теми, которые остались;
 - гипотеза различия возраста между теми клиентами, которые ушли и теми, которые остались.
6. Общий вывод и рекомендации.

## Загрузка данных

In [ ]:
# импортируем нужные нам в проекте библиотеки
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats as st


import warnings
warnings.filterwarnings('ignore')

In [ ]:
# загрузим данные и сохраним в переменной df
df = pd.read_csv('/datasets/bank_dataset.csv')

In [ ]:
df.head()

In [ ]:
df.info()

**Вывод:**

Датасет состоит из 12 столбцов и 10 000 строк. По столбцу `Balance` (баланс на карте) видны попуски - либо у клиентов нет денег на картах, либо связано с технической неполадкой выгрузки данных.

Необходимо разобраться со следующими моментами:
- привести названия колонок к нижнему регистру
- подумать над необходимостью заполнения пропусков
- проверить наличие дубликатов  
- заменить тип данных в столбцах `gender` и `city`, из категориальных в колличественные

## Предобработка данных

Приведем названия столбцов в нижнему регистру.

In [ ]:
df.columns = df.columns.str.lower()
df.head(1)

Займемся пропусками. 

In [ ]:
df.isna().sum()

3617 пропуска - это больше трети от всех значений. Посмотрим, как распределены пропуски в столбце баланс и попробуем найти зависимость.

In [ ]:
df[df['balance'].isna()]['city'].value_counts()

In [ ]:
df[df['balance'].isna()]['creditcard'].value_counts()

In [ ]:
df[df['balance'].isna()]['products'].value_counts()

In [ ]:
df[df['balance'].isna()]['churn'].value_counts()

Итак пропущенные значения баланса оказались:
- только двух городах (Ярославль и Рыбинск),
- у держателей кредитных карт пропусков больше в 2.5 раза, 
- у пользователей двух продуктов больше всего пропусков, 
- в категории "оставшихся в банке" клиентов в 6.2 раза больше пропущенных значений. 

По логике баланс может быть как положительным, нулевым, либо вообще отрицательным. Можно было бы заполнить пропуски нулями, но для этого следует уточнить природу их возникновения у тех, кто передал данные. 

В виду невозможности этого сделать, пропущенные значения оставим, как есть.

Проверим наличие дубликатов.

In [ ]:
df.duplicated().sum()

Дубликаты не обнаружены.

Для проведения дальнейшего корректного анализа, в том числе корреляционного, категориальные переменные в `gender` и `city` меняем на колличественные. 

In [ ]:
df.describe()

In [ ]:
# используем метод One-Hot Encoding
df = pd.get_dummies(df)
df

**Вывод:** 

* Названия столбцов приведены к единому регистру - нижнему.
* Пропуски обнаружены по столбцу `balance`, принято решение пока оставить как есть.
* Дубликаты не обнаружены.
* Заменены типы данных в столбцах `gender` и `city`.

## Исследовательский анализ данных EDA

### Анализ средних значений признаков и стандартного отклонения без разреза по оттоку

Посмотрим на распределение значений. Применим функцию `describe`.

In [ ]:
df.describe()

**Предварительный вывод**
* все клиенты уникальные, всего 10 000 чел.
* в среднем, кредитный рейтинг нелохой - 650 и практически равен медианному
* все клиенты совершеннолетние, возрастом от 18 до 92 лет, средний возраст 39 лет
* у большинства клиентов в собственности 5 объектов
* среднее количество денег на счете равно 119 тыс. 
* клиенты пользуются 1-2 продуктами банка
* большинство имеют кредитную карту
* активность проявляют чуть больше половины клиентов
* средний отток составляет 20%, что довольно много
* по географии распределения клиентов: половина всех живет в Ярославле, еще по четверти в Ростове Великом и Рыбинске
* мужчин немного больше - на 4.5%

Больше всего вопросов вызывает подозрительно низкая граница заработной платы клиентов. Рассмотрим поподробней этот момент.

In [ ]:
df.sort_values('estimated_salary').head(10)

Сразу заметим, что у большинства клиентов с самой низкой зарплатой показатель баланса либо примерно равен среднему (а то и выше), либо информация отсутсвует. Помимо этого у большинства клиентов оформлена кредитная карта. Что также же наводит на мысли о взаимосвязи пропусков.

### Анализ средних значений признаков для групп с оттоком и без

Рассчитаем средние значения признаков для ушедших и оставшихся.

In [ ]:
df.groupby('churn').mean()

**Предварительный вывод**

В целом можно сказать, что ушедшие клиенты мало, чем отличаются от тех, кто остался. Но различия все же присутствуют, вот наболее сильные:
* уходят в основном те, кто постарше
* активные клиенты чаще остаются 
* в Ростове отточных клиентов больше оставшихся, а вот в Рыбинске и Ярославле- наооборот
* среди ушедших клиентов - больше женщин

In [ ]:
numeric = ['score', 'age', 'balance', 'estimated_salary']
categorical = ['creditcard', 'loyalty', 'objects', 'products', 
               'city_Ростов Великий','city_Рыбинск', 'city_Ярославль', 'gender_Ж', 'gender_М' ]

In [ ]:
fig, ax = plt.subplots(2, 2, figsize=(20, 12))
ax = ax.flatten()
for col, subplot in zip(numeric, ax):
    sns.histplot(df, x=col, ax=subplot, hue="churn", element="step", stat="density", common_norm=False)
    subplot.set_ylabel('Количество')
    subplot.set_xlabel(col)
    subplot.set_title('Гистограмма распределения признака {}'. format(col), size=15)
fig.suptitle("Количество клиентов в разрезе категориальных переменных", size=20)

**Вывод:** 
* Чаще уходят из банка клиенты с более низким кредитным рейтингом, хотя в цифрах эта разница не значительна - в среднем это 6.5 кредитных балов.
* Чем старше клиент, тем больше вероятность его ухода - средний возраст 45 лет, против 37 лет у оставшихся.
* По балансу на счете нет сильной разницы. Разве что у остающихся клиентов график имеет более равномерное распределение.
* Зарплата у тех, кто уходит немного выше (~ 1700 рублей).

In [ ]:
fig, ax = plt.subplots(3,3, figsize=(16, 16))
ax = ax.flatten()
for col, subplot in zip(categorical, ax):
    sns.histplot(df, x=col, ax=subplot, hue="churn", element="step", stat="density", common_norm=False)
    subplot.set_ylabel('Количество')
    subplot.set_xlabel(col)
    subplot.set_title('Гистограмма распределения признака {}'. format(col), size=10)
fig.suptitle("Количество клиентов в разрезе колличественных переменных", size=20)

**Вывод:**
* Обе группы в равной мере пользуются кредитными картами.
* Среди оставшихся клиентов, доля активных больше - 55% против 45% неактивных. В группе ушедших ситуация другая - 34% активных против 64% неактивных.
* Ситуация более менее схожа. Чем большим количеством объектов владеют клиенты, там меньше вероятность их оттока.
* По количеству человек, чаще всего уходят клиенты, пользующиеся 1 продуктом. Но также можно сказать, что с вероятностью 90-100% уходят клиенты, у которых 3-4 продукта в пользовании. Менее всего подвержены оттоку клиенты с 2 продуктами.
* В Ростове Великом самая высокая динамика оттока клиентов.
* В Рыбинске, как и в Ярославле динамика удержания пока что в норме - отток не превышает удержание.
* Женщины более склонны уходить - почти 60% покидают банк.
* С мужчинами ситуация лучше - более 57% остаются.

### Корреляционный анализ

Проведем корреляционный анализ.

In [ ]:
plt.figure(figsize=(15,10))

sns.heatmap(df.corr(), annot=True, cmap="coolwarm")
plt.title('Матрица корреляций')
plt.show()

**Вывод:**
* Мультиколлинеарных признаков не наблюдается.
* Наиболее сильное влияние на отток оказывают:
    * возраст клиентов - `0.29`;
    * город (особенно Ростов Великий) - `0.17`;
    * активность клиентов -  `-0.16`; причем здесь наблюдается именно обратная корреляция, т.е. чем больше активность, тем меньше отток;
    * пол клиентов - `0.11` (одинаково для обоих гендеров); для женщин - прямая зависимость, а для мужчин обратная;
    
В общем и в целом нельзя сказать о какой-то сильной зависимости признаков. Тем не менее, стоит проверить возраст клиентов в дальнейшем при проверке гипоттез, как фактор влияния на отток.

## Портрет клиентов, склонных уходить из банка

На основе проведенного исследовательского анализа составим портреты клиентов, которые склонны уходить, а также тех, кто остается.

**Портрет клиентов, склонных к оттоку**
* В основном женщины;
* Средний возраст - 45 лет;
* Город проживания - Ростов Великий;
* Низкое проявление активности;
* Средняя зарплата 99 738 рублей;
* В среднем имеют меньше 5 объектов в собственности;
* Средний кредитный рейтинг 645;
* В основном пользуются 1 продуктом, плюс 100% отток у тех, кто пользуется 4 продуктами;
* Пользуются кредитными картами.

**Портрет клиентов, несклонных к оттоку**
* В основном мужчины;
* Средний возраст - 37 лет;
* Город проживания - Ярославль и Рыбинск;
* Высокое проявление активности;
* Средняя зарплата 101 465 рублей;
* В среднем имеют больше 5 объектов в собственности;
* Средний кредитный рейтинг 651;
* В основном пользуются 2 продуктами;
* Пользуются кредитными картами.

## Формулировка и проверка гипотез

### Гипотеза различия дохода между теми клиентами, которые ушли и теми, которые остались

**Нулевая гипотеза:** доход клиентов, которые ушли равен доходу оставшихся клиентов.

**Альтернативная гипотеза:** доход клиентов, которые ушли и доход оставшихся клиентов различаются. 

Поскольку мы имеем дело со средними, то применим метод проверки гипотезы о равенстве двух средних. Для проверки подойдёт t-test(тест Стьюдента).

In [ ]:
# подготовим выборки
df_churn_1 = df[df['churn']==1]['estimated_salary'] # для тех, кто ушел
df_churn_0 = df[df['churn']==0]['estimated_salary'] # для тех, кто остался

Определим равны ли выборки по количеству значений.

In [ ]:
print(
    F"Размеры выборок составляют:\n для попавших в отток - {df_churn_1.count()}"
    F"\n для оставшихся в банке - {df_churn_0.count()}"
)

Так как выбороки имеют разные размеры, установим значение параметра `equal_var` равное  `False`

In [ ]:
# зададим критический уровень статистической значимости равным 5%
alpha = 0.05

results = st.ttest_ind(df_churn_1, df_churn_0, equal_var = False)

print('p-значение:', results.pvalue)

if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу")
else:
    print("Не получилось отвергнуть нулевую гипотезу")

**Вывод:**

Не получилось отвергнуть нулевую гипотезу: нельзя сказать о том, что разница между средними значениями статистически значима.

### Гипотеза различия возраста между теми клиентами, которые ушли и теми, которые остались

**Нулевая гипотеза:** возраст клиентов, которые ушли равен возрасту оставшихся клиентов.

**Альтернативная гипотеза:** возраст клиентов, которые ушли и возраст оставшихся клиентов различаются. 

In [ ]:
# подготовим выборки
df_age_1 = df[df['churn']==1]['age'] # для тех, кто ушел
df_age_0 = df[df['churn']==0]['age'] # для тех, кто остался

Размер выборок для признака "возраст" равен размеру выборки для признака "доход", поэтому параметр `equal_var` оставляем  `False`

In [ ]:
alpha = 0.05

results = st.ttest_ind(df_age_1, df_age_0, equal_var = False)

print('p-значение:', results.pvalue)

if (results.pvalue < alpha):
    print("Отвергаем нулевую гипотезу.")
else:
    print("Не получилось отвергнуть нулевую гипотезу.")

**Вывод:**

Отвергаем нулевую гипотезу. Значит между средним возрастом ушедших и оставшихся клиентов имеется статистически значимая разница.

## Общий вывод и рекомендации

1. В процессе **загрузки и предобработки** было определено, что датасет состоит из 12 столбцов и 10 000 строк. Выполнены следующий действия:
* Названия столбцов приведены к единому регистру - нижнему.
* Пропуски, обнаруженные по столбцу `balance`, было принято решение не трогать.
* Заменены типы данных в столбцах `gender` и `city`.

2. В результате **исследовательского анализа EDA** были определены портреты клиентов:

**Портрет клиентов, склонных к оттоку**
* В основном женщины;
* Средний возраст - 45 лет;
* Город проживания - Ростов Великий;
* Низкое проявление активности;
* Средняя зарплата 99 738 рублей;
* В среднем имеют меньше 5 объектов в собственности;
* Средний кредитный рейтинг 645;
* В основном пользуются 1 продуктом, плюс 100% отток у тех, кто пользуется 4 продуктами;
* Пользуются кредитными картами.

**Портрет клиентов, несклонных к оттоку**
* В основном мужчины;
* Средний возраст - 37 лет;
* Город проживания - Ярославль и Рыбинск;
* Высокое проявление активности;
* Средняя зарплата 101 465 рублей;
* В среднем имеют больше 5 объектов в собственности;
* Средний кредитный рейтинг 651;
* В основном пользуются 2 продуктами;
* Пользуются кредитными картами.

3. В процессе **проверки гипотез**:

* не получилось отвергнуть гипотезу `о различии дохода между теми клиентами, которые ушли и теми, которые остались`, так как нельзя сказать, что разница между средними значениями статистически значима;
* была отвергнута гипотеза `о равенстве возраста ушедших и оставшихся клиентов`, что означает наличие статистически значимой разницы между средним возрастом ушедших и оставшихся клиентов.


**Рекомендации:**
1. В первую очередь обратить внимание на клиентов старшей возрастной группы. Как вариант, предлагать комплексные предложения, например дебетовые карты с повышенным кэшбеком в аптеках и на остаток баланса карты. Также будет неплохим подспорьем увеличить процентную ставку на вклады, т.к. пенсионеры склонны откладывать деньги.
2. Провести разбор оттока клиентов в городе Ростов Великий. Возможно банки-конкуренты предлагают лучшие условия обслуживания. 
3. Необходимо повысить активность клиентов. Как именно: внедрение различных бонусных программ при пользовании продуктами банка, оповещать об интересных событиях и акциях путем смс-информирования и push-уведомлений, возможно доработать приложение банка - повысить удобство пользования и качество работы.
4. Улучшить отдельные кредитные условия для клиентов-женщин. Возможно необходимо проработать смягчение требований к получению займов.